#TP3

Instalando as bibliotecas

In [2]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.1 MB/s 


In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Parte 1

In [21]:
import string
import unidecode
import math
import numpy
import re


def abreArquivo(nomeArquivo):
  fp = open(nomeArquivo, 'r')
  texto = fp.read()
  return texto

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, remove duplicatas e 
#retorna a lista de termos
def separaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)
  termos.sort()
  termos = list(dict.fromkeys(termos))

  return termos

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, mas não remove duplicatas
def listaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)

  return termos

def separaString(texto):
  texto = unidecode.unidecode(texto)
  pontuacao = string.punctuation
  for i in pontuacao:
    texto = texto.replace(i,"")
  
  texto = re.sub(r'\d+', '', texto)
  termos = texto.lower().split()

  return termos

def normalizaConsulta(vocabulario, consulta):
  termosConsulta = separaString(consulta)
  termos = []
  for i in termosConsulta:
    if i in vocabulario:
      termos.append(i)

  return termos
  
def comparaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    if i in documento:
      bagOfWords.append(1)
    else:
      bagOfWords.append(0)

  return bagOfWords

def frequenciaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    for j in documento:
      if j in vocabulario:
        contador = documento.count(i)
        bagOfWords.append(contador)
        break
      else:
        bagOfWords.append(0)
        break

  return bagOfWords

def calculaTF(bagOfWords):
  tf = []
  log = 0
  for i in bagOfWords:
    num = int(i)
    if num!=0:
      log = math.log(num, 2)
      tf.append(round(1+log, 3))
    else:
      tf.append(0)

  return tf

def calculaIDF(lista):
  vocabulario = geraVocabulario(lista)
  idf = []
  for i in vocabulario:
      contador = 0
      for j in lista:
          documento = separaArquivo(j)
          if i in documento:
              contador = contador + 1
      idf.append(contador)
  idf = [round(math.log(len(lista)/i, 2),3) for i in idf]

  return idf

def calculaTFIDFLista(vocabulario, lista):
  idf = calculaIDF(lista)
  listaTFIDF = []
  for i in lista:
    tfidf = []
    documento = listaArquivo(i)
    bagOfWords = frequenciaTermos(vocabulario, documento)
    tf = calculaTF(bagOfWords)
    tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]
    listaTFIDF.append(tfidf)
  
  return listaTFIDF

def calculaTFIDFConsulta(vocabulario, lista, consulta):
  termosConsulta = normalizaConsulta(vocabulario, consulta)
  bag = frequenciaTermos(vocabulario, termosConsulta)
  idf = calculaIDF(lista)
  tf = calculaTF(bag)
  tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]
  print(f'TF-IDF Consulta: {tfidf}')

  return tfidf
  

def mostraDadosListaArquivos(vocabulario, lista):
  for i in lista:
    bagOfWords = []
    documento = listaArquivo(i)
    print(f'\nDocumento: {documento}')
    bagOfWords = frequenciaTermos(vocabulario, documento)
    print(f'Bag of Words: {bagOfWords}')
    tf = calculaTF(bagOfWords)
    print(f'TF: {tf}')


def geraVocabulario(lista):
    vocabulario = []
    for i in lista:
        documento = separaArquivo(i)
        vocabulario = vocabulario + documento
    vocabulario = list(dict.fromkeys(vocabulario))
    vocabulario.sort()

    return vocabulario

def escreveVocabulario(lista):
    vocabulario = geraVocabulario(lista)
    fp = open('vocabulario.txt', 'w')
    for i in vocabulario:
        fp.write(i + '\n')

def calculaSimilaridade(vetorA, vetorB):
  produtoInterno = 0
  if len(vetorA)>0 and len(vetorB)>0: 
    for i in range (len(vetorA)):
      temp = vetorA[i]*vetorB[i]
      produtoInterno = produtoInterno + temp

  normaVetorA = numpy.linalg.norm(vetorA)
  normaVetorB = numpy.linalg.norm(vetorB)
  #print(f'Norma A: {normaVetorA}')
  #print(f'Norma B: {normaVetorB}')
  produtoNorma = normaVetorA*normaVetorB
  if produtoNorma!=0:
    similaridade = produtoInterno/produtoNorma
  else:
    similaridade = 0

  return similaridade

def calculaSimilaridadeLista(tfidf, vetorConsulta):
  vetorSimilaridades = []
  for i in range(len(tfidf)):
    vetorSimilaridades.append(calculaSimilaridade(tfidf[i], vetorConsulta))

  return vetorSimilaridades

def mostraVetor(vetor):
  print('\n'.join('{}: {}'.format(*k) for k in enumerate(vetor, 1)))

def mostraNomeVetor(vetor, lista):
  for i in range(len(vetor)):
    print(f'{lista[i]}: {vetor[i]}')

def mostraDocSimilar(vetorSimilaridade, lista):
  maiorSim = max(vetorSimilaridade)

  if maiorSim == 0:
    print(f'\nNao existe documento similar na colecao')
    return

  posDoc = vetorSimilaridade.index(maiorSim)
  print(f'\nDocumento mais Similar: {lista[posDoc]}')
  print(abreArquivo(lista[posDoc]))

def inputCalculaConsulta(vocabulario, lista, tfidf):
  while(1):
    consulta = input("\nDigite a consulta ('!' para terminar): ")
    if consulta == '!':
      break

    vetorConsulta = calculaTFIDFConsulta(vocabulario, lista, consulta)

    vetorSimilaridade = calculaSimilaridadeLista(tfidf, vetorConsulta)

    print('\nGrau de Similaridade')
    mostraNomeVetor(vetorSimilaridade, lista)

    mostraDocSimilar(vetorSimilaridade, lista)

  
listaDoc = ['d1.txt', 'd2.txt', 'd3.txt', 'd4.txt']

vocabulario = geraVocabulario(listaDoc)
print(f'Vocabulario: {vocabulario}')
mostraDadosListaArquivos(vocabulario, listaDoc)

idf = calculaIDF(listaDoc)
print(f'\nIDF: {idf}\n')

tfidf = calculaTFIDFLista(vocabulario, listaDoc)

print('TF-IDF')
mostraNomeVetor(tfidf, listaDoc)

inputCalculaConsulta(vocabulario, listaDoc, tfidf)

Vocabulario: ['am', 'be', 'da', 'do', 'i', 'is', 'it', 'let', 'not', 'or', 'therefore', 'think', 'to', 'what']

Documento: ['to', 'do', 'is', 'to', 'be', 'to', 'be', 'is', 'to', 'do']
Bag of Words: [0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0]
TF: [0, 2.0, 0, 2.0, 0, 2.0, 0, 0, 0, 0, 0, 0, 3.0, 0]

Documento: ['to', 'be', 'or', 'not', 'to', 'be', 'i', 'am', 'what', 'i', 'am']
Bag of Words: [2, 2, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 2, 1]
TF: [2.0, 2.0, 0, 0, 2.0, 0, 0, 0, 1.0, 1.0, 0, 0, 2.0, 1.0]

Documento: ['i', 'think', 'therefore', 'i', 'am', 'do', 'be', 'do', 'be', 'do']
Bag of Words: [1, 2, 0, 3, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0]
TF: [1.0, 2.0, 0, 2.585, 2.0, 0, 0, 0, 0, 0, 1.0, 1.0, 0, 0]

Documento: ['do', 'do', 'do', 'da', 'da', 'da', 'let', 'it', 'be', 'let', 'it', 'be']
Bag of Words: [0, 2, 3, 3, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0]
TF: [0, 2.0, 2.585, 2.585, 0, 0, 2.0, 2.0, 0, 0, 0, 0, 0, 0]

IDF: [1.0, 0.0, 2.0, 0.415, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0]

TF-IDF
d1.txt: [0.0, 0

Parte 2

In [ ]:
import string
import unidecode
import math
import numpy
import re
import nltk


def abreArquivo(nomeArquivo):
  fp = open(nomeArquivo, 'r')
  texto = fp.read()
  return texto

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, remove duplicatas e 
#retorna a lista de termos
def separaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)
  termos.sort()
  termos = list(dict.fromkeys(termos))

  return termos

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, mas não remove duplicatas
def listaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)

  return termos

#stopwords tratadas
def getStopwords():
  nltk.download('stopwords')
  stopwords = nltk.corpus.stopwords.words('portuguese')
  stopwordsUni = []
  for i in stopwords:
    stopwordsUni.append(unidecode.unidecode(i))
  stopwordsUni = list(dict.fromkeys(stopwordsUni))

  return stopwordsUni

stopwords = getStopwords()
def separaString(texto):
  texto = unidecode.unidecode(texto)
  pontuacao = string.punctuation
  for i in pontuacao:
    texto = texto.replace(i,"")
  
  texto = re.sub(r'\d+', '', texto)
  termos = texto.lower().split()

  for i in stopwords:
    while i in termos:
      termos.remove(i)

  return termos

def geraVocabulario(lista):
    vocabulario = []
    for i in lista:
        documento = separaArquivo(i)
        vocabulario = vocabulario + documento
    vocabulario = list(dict.fromkeys(vocabulario))
    vocabulario.sort()

    return vocabulario

def escreveVocabulario(lista):
    vocabulario = geraVocabulario(lista)
    fp = open('vocabulario.txt', 'w')
    for i in vocabulario:
        fp.write(i + '\n')

#teste com alguns hinos
listaDoc = ['hinoAtleticoMG.txt', 'hinoCorinthians.txt', 'hinoFlamengo.txt', 'hinoFluminense.txt',
            'hinoInternacional.txt', 'hinoPalmeiras.txt', 'hinoSantos.txt', 'hinoSaoPaulo.txt']


vocabulario = geraVocabulario(listaDoc)

print(f'StopWords: {stopwords}')
print(f'\nVocabulario: {vocabulario}')

StopWords: ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'ja', 'eu', 'tambem', 'so', 'pelo', 'pela', 'ate', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'voce', 'essa', 'num', 'nem', 'suas', 'meu', 'minha', 'numa', 'pelos', 'elas', 'qual', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'voces', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estiveramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivessemos', 'estivessem', 'estiver', 'estivermos', 'estiverem',

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Parte 3 sem steamming

In [29]:
import string
import unidecode
import math
import numpy
import re
import nltk


def abreArquivo(nomeArquivo):
  fp = open(nomeArquivo, 'r')
  texto = fp.read()
  return texto

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, remove duplicatas e 
#retorna a lista de termos
def separaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)
  termos.sort()
  termos = list(dict.fromkeys(termos))

  return termos

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, mas não remove duplicatas
def listaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)

  return termos

#stopwords tratadas
def getStopwords():
  nltk.download('stopwords')
  stopwords = nltk.corpus.stopwords.words('portuguese')
  stopwordsUni = []
  for i in stopwords:
    stopwordsUni.append(unidecode.unidecode(i))
  stopwordsUni = list(dict.fromkeys(stopwordsUni))

  return stopwordsUni

stopwords = getStopwords()
def separaString(texto):
  texto = unidecode.unidecode(texto)
  pontuacao = string.punctuation
  for i in pontuacao:
    texto = texto.replace(i,"")
  
  texto = re.sub(r'\d+', '', texto)
  termos = texto.lower().split()

  for i in stopwords:
    while i in termos:
      termos.remove(i)

  return termos

def normalizaConsulta(vocabulario, consulta):
  termosConsulta = separaString(consulta)
  termos = []
  for i in termosConsulta:
    if i in vocabulario:
      termos.append(i)

  return termos

  
def comparaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    if i in documento:
      bagOfWords.append(1)
    else:
      bagOfWords.append(0)

  return bagOfWords

def frequenciaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    for j in documento:
      if j in vocabulario:
        contador = documento.count(i)
        bagOfWords.append(contador)
        break
      else:
        bagOfWords.append(0)
        break

  return bagOfWords

def calculaTF(bagOfWords):
  tf = []
  log = 0
  for i in bagOfWords:
    num = int(i)
    if num!=0:
      log = math.log(num, 2)
      tf.append(round(1+log, 3))
    else:
      tf.append(0)

  return tf

def calculaIDF(lista):
  vocabulario = geraVocabulario(lista)
  idf = []
  for i in vocabulario:
      contador = 0
      for j in lista:
          documento = separaArquivo(j)
          if i in documento:
              contador = contador + 1
      idf.append(contador)
  idf = [round(math.log(len(lista)/i, 2),3) for i in idf]

  return idf

def calculaTFIDFLista(vocabulario, lista):
  idf = calculaIDF(lista)
  listaTFIDF = []
  for i in lista:
    tfidf = []
    documento = listaArquivo(i)
    bagOfWords = frequenciaTermos(vocabulario, documento)
    tf = calculaTF(bagOfWords)
    tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]
    listaTFIDF.append(tfidf)
  
  return listaTFIDF

def calculaTFIDFConsulta(vocabulario, lista, consulta):
  termosConsulta = normalizaConsulta(vocabulario, consulta)
  bag = frequenciaTermos(vocabulario, termosConsulta)
  idf = calculaIDF(lista)
  tf = calculaTF(bag)
  tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]

  return tfidf
  

def mostraDadosListaArquivos(vocabulario, lista):
  for i in lista:
    bagOfWords = []
    documento = listaArquivo(i)
    print(f'\nDocumento: {documento}')
    bagOfWords = frequenciaTermos(vocabulario, documento)
    print(f'Bag of Words: {bagOfWords}')
    tf = calculaTF(bagOfWords)
    print(f'TF: {tf}')


def geraVocabulario(lista):
    vocabulario = []
    for i in lista:
        documento = separaArquivo(i)
        vocabulario = vocabulario + documento
    vocabulario = list(dict.fromkeys(vocabulario))
    vocabulario.sort()

    return vocabulario

def escreveVocabulario(lista):
    vocabulario = geraVocabulario(lista)
    fp = open('vocabulario.txt', 'w')
    for i in vocabulario:
        fp.write(i + '\n')

def calculaSimilaridade(vetorA, vetorB):
  produtoInterno = 0
  for i in range (len(vetorA)):
    temp = vetorA[i]*vetorB[i]
    produtoInterno = produtoInterno + temp

  normaVetorA = numpy.linalg.norm(vetorA)
  normaVetorB = numpy.linalg.norm(vetorB)
  produtoNorma = normaVetorA*normaVetorB
  if produtoNorma!=0:
    similaridade = produtoInterno/produtoNorma
  else:
    similaridade = 0

  return similaridade

def calculaSimilaridadeLista(tfidf, vetorConsulta):
  vetorSimilaridades = []
  for i in range(len(tfidf)):
    vetorSimilaridades.append(calculaSimilaridade(tfidf[i], vetorConsulta))

  return vetorSimilaridades

def mostraVetor(vetor):
  print('\n'.join('{}: {}'.format(*k) for k in enumerate(vetor, 1)))

def mostraNomeVetor(vetor, lista):
  for i in range(len(vetor)):
    print(f'{lista[i]}: {vetor[i]}')

def mostraDocSimilar(vetorSimilaridade, lista):
  maiorSim = max(vetorSimilaridade)

  if maiorSim == 0:
    print(f'\nNao existe documento similar na colecao')
    return

  posDoc = vetorSimilaridade.index(maiorSim)
  print(f'\nDocumento mais Similar: {lista[posDoc]}')
  print(abreArquivo(lista[posDoc]))

def inputCalculaConsulta(vocabulario, lista, tfidf):
  while(1):
    consulta = input("\nDigite a consulta ('!' para terminar): ")
    if consulta == '!':
      break

    vetorConsulta = calculaTFIDFConsulta(vocabulario, lista, consulta)

    vetorSimilaridade = calculaSimilaridadeLista(tfidf, vetorConsulta)

    print('\nGrau de Similaridade')
    mostraNomeVetor(vetorSimilaridade, lista)

    mostraDocSimilar(vetorSimilaridade, lista)




listaDoc = [ 'hinoAmericaMG.txt', 'hinoAthletico.txt', 'hinoAtleticoGO.txt', 'hinoAtleticoMG.txt', 'hinoAvai.txt',
          'hinoBotafogo.txt', 'hinoBragantino.txt', 'hinoCeara.txt', 'hinoCorinthians.txt', 'hinoCoritiba.txt',
          'hinoCuiaba.txt', 'hinoFlamengo.txt', 'hinoFluminense.txt', 'hinoFortaleza.txt', 'hinoGoias.txt',
          'hinoInternacional.txt', 'hinoJuventude.txt','hinoPalmeiras.txt', 'hinoSantos.txt', 'hinoSaoPaulo.txt'] 
vocabulario = geraVocabulario(listaDoc)

tfidf = calculaTFIDFLista(vocabulario, listaDoc)
inputCalculaConsulta(vocabulario, listaDoc, tfidf)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Digite a consulta ('!' para terminar): salve o tricolor  paulista amado clube brasileiro

Grau de Similaridade
hinoAmericaMG.txt: 0.0
hinoAthletico.txt: 0.0
hinoAtleticoGO.txt: 0.0
hinoAtleticoMG.txt: 0.018898852709640757
hinoAvai.txt: 0.0
hinoBotafogo.txt: 0.0
hinoBragantino.txt: 0.0
hinoCeara.txt: 0.0
hinoCorinthians.txt: 0.10682467878266919
hinoCoritiba.txt: 0.0
hinoCuiaba.txt: 0.00941256448949808
hinoFlamengo.txt: 0.06690606288519169
hinoFluminense.txt: 0.13636071958696327
hinoFortaleza.txt: 0.11373049509712517
hinoGoias.txt: 0.014948054713066497
hinoInternacional.txt: 0.004184533865584773
hinoJuventude.txt: 0.022046048889236804
hinoPalmeiras.txt: 0.04000144662257495
hinoSantos.txt: 0.025338820524713493
hinoSaoPaulo.txt: 0.44237159595981657

Documento mais Similar: hinoSaoPaulo.txt
Salve o tricolor paulista
Amado clube brasileiro
Tu és forte, tu és grande
Dentre os grandes és o primeiro
Tu és forte, tu és grande
Dentre os grandes és o primeiro
Ó tricolor
Clube bem amado
As tuas gl

Parte 3 com steamming

In [28]:
import string
import unidecode
import math
import numpy
import re
import nltk
from nltk.stem import PorterStemmer


def abreArquivo(nomeArquivo):
  fp = open(nomeArquivo, 'r')
  texto = fp.read()
  return texto

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, remove duplicatas e 
#retorna a lista de termos
def separaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)
  termos.sort()
  termos = list(dict.fromkeys(termos))

  return termos

#abre o arquivo, faz o unidecode, remove pontuação, separa os termos, ordena os termos, mas não remove duplicatas
def listaArquivo(nomeArquivo):
  texto = abreArquivo(nomeArquivo)
  termos = separaString(texto)

  return termos

#stopwords tratadas
def getStopwords():
  nltk.download('stopwords')
  stopwords = nltk.corpus.stopwords.words('portuguese')
  stopwordsUni = []
  for i in stopwords:
    stopwordsUni.append(unidecode.unidecode(i))
  stopwordsUni = list(dict.fromkeys(stopwordsUni))

  return stopwordsUni


#variaveis globais utiliadas no separaString
stopwords = getStopwords()
stemmer = PorterStemmer()

def separaString(texto):
  texto = unidecode.unidecode(texto)
  pontuacao = string.punctuation
  for i in pontuacao:
    texto = texto.replace(i,"")
  
  texto = re.sub(r'\d+', '', texto)
  termos = texto.lower().split()

  for i in stopwords:
    while i in termos:
      termos.remove(i)

  termosRad = []
  for i in range(len(termos)):
    termosRad.append(stemmer.stem(termos[i]))

  return termosRad

def normalizaConsulta(vocabulario, consulta):
  termosConsulta = separaString(consulta)
  termos = []
  for i in termosConsulta:
    if i in vocabulario:
      termos.append(i)

  return termos

  
def comparaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    if i in documento:
      bagOfWords.append(1)
    else:
      bagOfWords.append(0)

  return bagOfWords

def frequenciaTermos(vocabulario, documento):
  bagOfWords = []
  for i in vocabulario:
    for j in documento:
      if j in vocabulario:
        contador = documento.count(i)
        bagOfWords.append(contador)
        break
      else:
        bagOfWords.append(0)
        break

  return bagOfWords

def calculaTF(bagOfWords):
  tf = []
  log = 0
  for i in bagOfWords:
    num = int(i)
    if num!=0:
      log = math.log(num, 2)
      tf.append(round(1+log, 3))
    else:
      tf.append(0)

  return tf

def calculaIDF(lista):
  vocabulario = geraVocabulario(lista)
  idf = []
  for i in vocabulario:
      contador = 0
      for j in lista:
          documento = separaArquivo(j)
          if i in documento:
              contador = contador + 1
      idf.append(contador)
  idf = [round(math.log(len(lista)/i, 2),3) for i in idf]

  return idf

def calculaTFIDFLista(vocabulario, lista):
  idf = calculaIDF(lista)
  listaTFIDF = []
  for i in lista:
    tfidf = []
    documento = listaArquivo(i)
    bagOfWords = frequenciaTermos(vocabulario, documento)
    tf = calculaTF(bagOfWords)
    tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]
    listaTFIDF.append(tfidf)
  
  return listaTFIDF

def calculaTFIDFConsulta(vocabulario, lista, consulta):
  termosConsulta = normalizaConsulta(vocabulario, consulta)
  bag = frequenciaTermos(vocabulario, termosConsulta)
  idf = calculaIDF(lista)
  tf = calculaTF(bag)
  tfidf = [round(tf[j]*idf[j], 3) for j in range(len(tf))]

  return tfidf
  

def mostraDadosListaArquivos(vocabulario, lista):
  for i in lista:
    bagOfWords = []
    documento = listaArquivo(i)
    print(f'\nDocumento: {documento}')
    bagOfWords = frequenciaTermos(vocabulario, documento)
    print(f'Bag of Words: {bagOfWords}')
    tf = calculaTF(bagOfWords)
    print(f'TF: {tf}')


def geraVocabulario(lista):
    vocabulario = []
    for i in lista:
        documento = separaArquivo(i)
        vocabulario = vocabulario + documento
    vocabulario = list(dict.fromkeys(vocabulario))
    vocabulario.sort()

    return vocabulario

def escreveVocabulario(lista):
    vocabulario = geraVocabulario(lista)
    fp = open('vocabulario.txt', 'w')
    for i in vocabulario:
        fp.write(i + '\n')

def calculaSimilaridade(vetorA, vetorB):
  produtoInterno = 0
  for i in range (len(vetorA)):
    temp = vetorA[i]*vetorB[i]
    produtoInterno = produtoInterno + temp

  normaVetorA = numpy.linalg.norm(vetorA)
  normaVetorB = numpy.linalg.norm(vetorB)
  produtoNorma = normaVetorA*normaVetorB
  if produtoNorma!=0:
    similaridade = produtoInterno/produtoNorma
  else:
    similaridade = 0

  return similaridade

def calculaSimilaridadeLista(tfidf, vetorConsulta):
  vetorSimilaridades = []
  for i in range(len(tfidf)):
    vetorSimilaridades.append(calculaSimilaridade(tfidf[i], vetorConsulta))

  return vetorSimilaridades

def mostraVetor(vetor):
  print('\n'.join('{}: {}'.format(*k) for k in enumerate(vetor, 1)))

def mostraNomeVetor(vetor, lista):
  for i in range(len(vetor)):
    print(f'{lista[i]}: {vetor[i]}')

def mostraDocSimilar(vetorSimilaridade, lista):
  maiorSim = max(vetorSimilaridade)

  if maiorSim == 0:
    print(f'\nNao existe documento similar na colecao')
    return

  posDoc = vetorSimilaridade.index(maiorSim)
  print(f'\nDocumento mais Similar: {lista[posDoc]}')
  print(abreArquivo(lista[posDoc]))

def inputCalculaConsulta(vocabulario, lista, tfidf):
  while(1):
    consulta = input("\nDigite a consulta ('!' para terminar): ")
    if consulta == '!':
      break

    vetorConsulta = calculaTFIDFConsulta(vocabulario, lista, consulta)

    vetorSimilaridade = calculaSimilaridadeLista(tfidf, vetorConsulta)

    print('\nGrau de Similaridade')
    mostraNomeVetor(vetorSimilaridade, lista)

    mostraDocSimilar(vetorSimilaridade, lista)




listaDoc = [ 'hinoAmericaMG.txt', 'hinoAthletico.txt', 'hinoAtleticoGO.txt', 'hinoAtleticoMG.txt', 'hinoAvai.txt',
          'hinoBotafogo.txt', 'hinoBragantino.txt', 'hinoCeara.txt', 'hinoCorinthians.txt', 'hinoCoritiba.txt',
          'hinoCuiaba.txt', 'hinoFlamengo.txt', 'hinoFluminense.txt', 'hinoFortaleza.txt', 'hinoGoias.txt',
          'hinoInternacional.txt', 'hinoJuventude.txt','hinoPalmeiras.txt', 'hinoSantos.txt', 'hinoSaoPaulo.txt'] 
vocabulario = geraVocabulario(listaDoc)

tfidf = calculaTFIDFLista(vocabulario, listaDoc)
inputCalculaConsulta(vocabulario, listaDoc, tfidf)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Digite a consulta ('!' para terminar): salve o tricolor  paulista amado clube brasileiro

Grau de Similaridade
hinoAmericaMG.txt: 0.0
hinoAthletico.txt: 0.0
hinoAtleticoGO.txt: 0.0
hinoAtleticoMG.txt: 0.019427766446752464
hinoAvai.txt: 0.0
hinoBotafogo.txt: 0.0
hinoBragantino.txt: 0.0
hinoCeara.txt: 0.0
hinoCorinthians.txt: 0.0994213614353375
hinoCoritiba.txt: 0.02703400052369766
hinoCuiaba.txt: 0.009644403967708283
hinoFlamengo.txt: 0.0688057522242013
hinoFluminense.txt: 0.13957246843072185
hinoFortaleza.txt: 0.11671283947014237
hinoGoias.txt: 0.015369710385788495
hinoInternacional.txt: 0.004340386047125665
hinoJuventude.txt: 0.023419556316695535
hinoPalmeiras.txt: 0.029704386173678293
hinoSantos.txt: 0.02634669891878538
hinoSaoPaulo.txt: 0.47855612522345864

Documento mais Similar: hinoSaoPaulo.txt
Salve o tricolor paulista
Amado clube brasileiro
Tu és forte, tu és grande
Dentre os grandes és o primeiro
Tu és forte, tu és grande
Dentre os grandes és o primeiro
Ó tricolor
Clube bem a